![](hacker-news-posts.jpg)

# Explorando as postagens no site Hacker News

Nesse projeto vamos trabalhar analisando as postagens de um dos principais sites de tecnologia, o [Hacker News](https://news.ycombinator.com/).

Você pode encontrar o dataset completo que vamos trabalhar nesse projeto [aqui](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts). O dataset possui mais de 300 mil linhas, mas para esse projeto iremos trabalhar com uma amostra reduzida de 20 mil linhas, onde removemos as postagens que não tiveram comentários.

Para essa análise estamos interessados especificamente em posts com títulos que começam com `Ask HN` ou `Show HN`. O `Ask HN` é utilizado para perguntar algo em específico para a comunidade e já o `Show HN` é usado para os usários compartilharem algum projeto, produto ou algo que achem interessante.

Dessa forma, iremos comparar esses dois tipos de postagens para responder às seguintes perguntas:
* Qual dos dois tipos recebem em média mais comentários?
* Postagens criadas em determinados períodos podem receber mais comentários?

---

## Conhecendo os dados

Primeiramente vamos ler o arquivo CSV com os dados e assim transformar em uma lista de listas.

In [1]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)

In [2]:
# Mostrar as 5 primeiras linhas
hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

### Eliminando a linha de cabeçalho

Para analisar os dados, precisamos primeiramente remover a primeira linha do dataset, que contém o cabeçalho. Assim vamos ter somente os dados que iremos trabalhar.

In [3]:
headers = hn[0] # salvando o cabeçalho
hn = hn[1:] # removendo o cabeçalho do dataset

In [4]:
headers # mostrar o cabeçalho

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

In [5]:
hn[:5] # verificando se o cabeçalho foi removido com sucesso

[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

---

## Filtrando os dados

Vamos iniciar aqui o processo filtrar os dados que começam com `Ask HN` e `Show HN`, que lembrando, são os dois tipos de posts que nos interessam.

Para isso, iremos separar em duas listas diferentes.

In [6]:
ask_posts = list()
show_posts = list()
other_posts = list()

for row in hn:
    title = row[1] # o título fica na segunda coluna
    
    # inserindo os dois tipos de postagens em cada lista
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

In [7]:
# Checagem da quantidade de posts em cada lista
print(f'Número de posts na lista ask_posts: {len(ask_posts)}')
print(f'Número de posts na lista show_posts: {len(show_posts)}')
print(f'Número de posts na lista other_posts: {len(other_posts)}')

Número de posts na lista ask_posts: 1744
Número de posts na lista show_posts: 1162
Número de posts na lista other_posts: 17194


In [8]:
# Checagem do conteúdo na lista ask_posts
ask_posts[:5]

[['12296411',
  'Ask HN: How to improve my personal website?',
  '',
  '2',
  '6',
  'ahmedbaracat',
  '8/16/2016 9:55'],
 ['10610020',
  'Ask HN: Am I the only one outraged by Twitter shutting down share counts?',
  '',
  '28',
  '29',
  'tkfx',
  '11/22/2015 13:43'],
 ['11610310',
  'Ask HN: Aby recent changes to CSS that broke mobile?',
  '',
  '1',
  '1',
  'polskibus',
  '5/2/2016 10:14'],
 ['12210105',
  'Ask HN: Looking for Employee #3 How do I do it?',
  '',
  '1',
  '3',
  'sph130',
  '8/2/2016 14:20'],
 ['10394168',
  'Ask HN: Someone offered to buy my browser extension from me. What now?',
  '',
  '28',
  '17',
  'roykolak',
  '10/15/2015 16:38']]

In [9]:
# Checagem do conteúdo na lista show_posts
show_posts[:5]

[['10627194',
  'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform',
  'https://iot.seeed.cc',
  '26',
  '22',
  'kfihihc',
  '11/25/2015 14:03'],
 ['10646440',
  'Show HN: Something pointless I made',
  'http://dn.ht/picklecat/',
  '747',
  '102',
  'dhotson',
  '11/29/2015 22:46'],
 ['11590768',
  'Show HN: Shanhu.io, a programming playground powered by e8vm',
  'https://shanhu.io',
  '1',
  '1',
  'h8liu',
  '4/28/2016 18:05'],
 ['12178806',
  'Show HN: Webscope  Easy way for web developers to communicate with Clients',
  'http://webscopeapp.com',
  '3',
  '3',
  'fastbrick',
  '7/28/2016 7:11'],
 ['10872799',
  'Show HN: GeoScreenshot  Easily test Geo-IP based web pages',
  'https://www.geoscreenshot.com/',
  '1',
  '9',
  'kpsychwave',
  '1/9/2016 20:45']]

In [10]:
# Checagem do conteúdo na lista other_posts
other_posts[:5]

[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

---

## Média de comentários por tipo de postagem

O passo agora é determinar se na média temos maior quantidade de postagens para o tipo `Ask HN` ou `Show HN`.

### Ask HN

In [11]:
# Número total de posts do tipo Ask HN
total_ask_comments = 0

for row in ask_posts:
    total_ask_comments += int(row[4]) # o número de comentários tá na coluna 5

In [12]:
print(f'Total de comentários do tipo Ask HN: {total_ask_comments}')

Total de comentários do tipo Ask HN: 24483


In [13]:
# Média de comentários por post
avg_ask_comments = total_ask_comments / len(ask_posts)
print(f'Média de comentários por post do tipo Ask HN: {avg_ask_comments:.2f}')

Média de comentários por post do tipo Ask HN: 14.04


### Show HN

In [14]:
# Número total de posts do tipo Show HN
total_show_comments = 0

for row in show_posts:
    total_show_comments += int(row[4]) # o número de comentários tá na coluna 5

In [15]:
print(f'Total de comentários do tipo Show HN: {total_show_comments}')

Total de comentários do tipo Show HN: 11988


In [16]:
# Média de comentários por post
avg_show_comments = total_show_comments / len(show_posts)
print(f'Média de comentários por post do tipo Show HN: {avg_show_comments:.2f}')

Média de comentários por post do tipo Show HN: 10.32


### Qual dos dois tipos recebem em média mais comentários?

Verificamos acima que a média de comentários por post do tipo `Ask HN` é superior a do `Show HN`, sendo aproximadamente __14 contra 10__ respectivamente. Dessa forma, vamos continuar nossa análise focando apenas no tipo __`Ask HN`__, por estarem mais propensas a receber comentários.

---

## Categorizando o número de postagens e comentários pela Hora Criada

### Quantidade de posts e comentários por hora

Nessa parte iremos calcular o número de posts criados por hora, juntamente com o total de comentários.

In [17]:
# Coletando a data de criação de cada postagem e a quantidade de comentários
import datetime as dt

result_list = list()

for row in ask_posts:
    result_list.append([row[6], int(row[4])]) # a coluna sete contém a info de data de criação

In [18]:
# Verificando se a lista foi preenchida com os elementos corretos
result_list[:10]

[['8/16/2016 9:55', 6],
 ['11/22/2015 13:43', 29],
 ['5/2/2016 10:14', 1],
 ['8/2/2016 14:20', 3],
 ['10/15/2015 16:38', 17],
 ['9/26/2015 23:23', 1],
 ['4/22/2016 12:24', 4],
 ['11/16/2015 9:22', 1],
 ['2/24/2016 17:57', 1],
 ['6/4/2016 17:17', 2]]

In [19]:
counts_by_hour = dict()
comments_by_hour = dict()

for row in result_list:
    date_info = row[0]
    date_info_dt = dt.datetime.strptime(date_info, '%m/%d/%Y %H:%M')
    hour = date_info_dt.strftime('%H')
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]

In [20]:
# Quantidade de postagens por hora
counts_by_hour

{'09': 45,
 '13': 85,
 '10': 59,
 '14': 107,
 '16': 108,
 '23': 68,
 '12': 73,
 '17': 100,
 '15': 116,
 '21': 109,
 '20': 80,
 '02': 58,
 '18': 109,
 '03': 54,
 '05': 46,
 '19': 110,
 '01': 60,
 '22': 71,
 '08': 48,
 '04': 47,
 '00': 55,
 '06': 44,
 '07': 34,
 '11': 58}

In [21]:
# Quantidade de comentários por hora
comments_by_hour

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

### Média de comentários por hora

Com a quantidade de comentários por hora, é possível fazer o cálculo da média de comentários em cada hora. A lista __counts_by_hour__ será utilizada nessa etapa. Uma nova lista vai conter a média/hora.

In [22]:
avg_by_hour = list()

for hour in comments_by_hour:
    avg_by_hour.append([hour, (comments_by_hour[hour] / counts_by_hour[hour])])

In [23]:
# Checagem da média
for hour in avg_by_hour:
    print(f'{hour[0]}: {hour[1]:.2f}')

09: 5.58
13: 14.74
10: 13.44
14: 13.23
16: 16.80
23: 7.99
12: 9.41
17: 11.46
15: 38.59
21: 16.01
20: 21.52
02: 23.81
18: 13.20
03: 7.80
05: 10.09
19: 10.80
01: 11.38
22: 6.75
08: 10.25
04: 7.17
00: 8.13
06: 9.02
07: 7.85
11: 11.05


### Ordenando pelas horas com maior média

Apesar de já termos os resultados que precisamos, esse formato dificulta a identificação das horas com os valores mais altos. Vamos terminar classificando os cinco valores mais altos em um formato mais fácil de ler.

In [24]:
# Deixando a média como primeiro elemento
swap_avg_by_hour = list()

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

In [25]:
swap_avg_by_hour

[[5.5777777777777775, '09'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [16.796296296296298, '16'],
 [7.985294117647059, '23'],
 [9.41095890410959, '12'],
 [11.46, '17'],
 [38.5948275862069, '15'],
 [16.009174311926607, '21'],
 [21.525, '20'],
 [23.810344827586206, '02'],
 [13.20183486238532, '18'],
 [7.796296296296297, '03'],
 [10.08695652173913, '05'],
 [10.8, '19'],
 [11.383333333333333, '01'],
 [6.746478873239437, '22'],
 [10.25, '08'],
 [7.170212765957447, '04'],
 [8.127272727272727, '00'],
 [9.022727272727273, '06'],
 [7.852941176470588, '07'],
 [11.051724137931034, '11']]

In [26]:
# Deixando a lista ordenada pelas média com mais alto valor
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

### Postagens criadas em determinados períodos podem receber mais comentários?

In [27]:
print('Top 5 Horas para postagens do tipo Ask HN:\n')

for row in sorted_swap[:5]:
    print(f'{row[1]}h: {row[0]:.2f} média de comentários por post')

Top 5 Horas para postagens do tipo Ask HN:

15h: 38.59 média de comentários por post
02h: 23.81 média de comentários por post
20h: 21.52 média de comentários por post
16h: 16.80 média de comentários por post
21h: 16.01 média de comentários por post


__15h__ é a hora que recebe em média mais comentários, com __38.59__ comentários por post. Isso é em torno de 60% a mais que a média das 2h, que é a segunda com maior média __(23.81)__.

---

## Conclusão

Nesse projeto analisamos as postagens do tipo `Ask HN` e `Show HN` para determinar qual tipo e horário recebem em média mais comentários. Com base em nossa análise, para maximizar a quantidade de comentários que uma postagem recebe, __recomendamos que a postagem seja do tipo `Ask HN` e criada entre 15h e 16h__.

Devemos levar em consideração também que o dataset que analisamos excluiu postagens sem comentários. Com isso, é mais correto dizer que dos posts que receberam comentários, os do tipo `Ask HN` receberam mais comentários em média e os que foram criados entre 15h e 16h tiveram o maioria dos comentários.

![](bot-reps-dataquest.jpg)